# <span style="color:darkblue">03-Spot_decomposition</span>

In this notebook we will guide you through performing spot decomposition using the BigFISH libray, opening your images in Napari, and saving the generated decomposed spot detection data. We decompose Transcription sites (Txs) within the cell nucleus to estimate the number of nascent mRNAs. For more documentation of the BigFISH library see: https://big-fish.readthedocs.io/en/stable/index.html. At the bottom of this notebook batch decomposition can be performed.

## 3.0 - Load libraries

In [ ]:
import re
from glob import glob
from skimage import io
from pathlib import Path
import numpy as np
import napari
import matplotlib.pyplot as plt
from bigfish.detection import decompose_dense
from bigfish.stack import remove_background_gaussian

import sys
sys.path.append('../')
from src.misc import group_experiments, load_data

***

## 3.1 - Example of spot decomposition on a single image

In [ ]:
# load image data
RNAs = io.imread('../data/*/CET111_CLB2Q670_SPIDER37_CY5_01.tif')
DAPI = io.imread('../data/*/CET111_CLB2Q670_SPIDER37_DAPI_01.tif')
DIC = io.imread('../data/*/CET111_CLB2Q670_SPIDER37_DIC_01.tif')
mask = io.imread('../data/*/Masks/CET111_CLB2Q670_SPIDER37_DIC_01_seg.tif')
nuclear_mask = io.imread('../data/*/Masks/MAX_CET111_CLB2Q670_SPIDER37_DAPI_01_seg.tif')

# load spot data
spot_data = np.load(glob('../data/*/Spots/CET111_CLB2Q670_SPIDER37_CY5_01_spots_thr*.npy')[0])

Add the loaded data to a Napari viewer and inspect whether the data is loaded correctly.

In [ ]:
viewer=napari.Viewer()

#scale
scale = (200,65,65)

viewer.add_image(RNAs, name='RNA channel',scale=scale)
viewer.add_image(DAPI, name='DAPI channel',scale=scale)

# parameter for guassian filtering
sigma = (0.75,2.3,2.3)
# guassian filtering
filt_RNAs=remove_background_gaussian(RNAs, sigma=sigma)

viewer.add_image(filt_RNAs, name='filt RNA channel',scale=scale)
viewer.add_labels(nuclear_mask, name='nuclei',scale=scale[-2:])
viewer.add_points(spot_data,scale=scale)

Decompose dense regions into seperate spots.

In [ ]:
# parameters
spot_radius = (1250, 170, 170)

In [ ]:
spots, dense_regions, reference_spots = decompose_dense(
    viewer.layers['filt RNA channel'].data,
    spot_data,
    voxel_size=scale,
    spot_radius=spot_radius,
    alpha=0.5,beta=2,gamma=1
)

Inspect whether the reference spot looks approximately gaussian.

In [ ]:
fig,axis=plt.subplots(ncols=reference_spots.shape[0],figsize=(20,10))

for i,ax in zip(range(len(axis)),axis):
    ax.imshow(reference_spots[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Inspect the detected dense RNA regions in Napari. Make sure all TXs in the nuclei are correctly identified. 

Note: false Txs identified outside of the cell nucleus will be filtered out in the next notebook.

In [ ]:
viewer.add_points(
    dense_regions[:,:3],
    scale=scale,
    edge_color='blue',
    name='Txs'
)

Number of Nascent RNAs per potential Txs.

In [ ]:
fig, ax = plt.subplots(facecolor='white')
ax.hist(dense_regions[:, 3])
ax.set_xlabel('RNas per Tx')
ax.set_ylabel('counts')
ax.set_xticks(range(0, max(dense_regions[:, 3] + 1), 1));

***

## 3.2 - Batch decomposition

In [ ]:
# paths
root_dir = '../data/zipped_example_data_smFISH_C.albicans'

#parameters - adjust if necessary
scale = (200, 65, 65)
spot_radius = (1250, 170, 170)
sigma=(0.75, 2.3, 2.3)
patch_size = (200, 200)

In [ ]:
experiments = group_experiments(root_dir)

print('I found the following experiments:')
print(experiments.keys())
print('select applicable experiments')

In [ ]:
experiments_to_process = ['CET111_CLB2Q670_SPIDER37','CET111_EFG1Q670_SPIDER37']

for identifier in experiments_to_process:
    fovs = experiments[identifier]
    
    for fov, paths, in fovs.items():
        print(f'processing {identifier=}, {fov=}')
        data = load_data(paths)
        spot_file_name = Path(paths['spots']).stem
        save_path = f"{root_dir}/Spots decomposition/{spot_file_name}"
        
        process = True
        # check if all files required for this step have been loaded
        for entry in ['spots', 'CY5']:
            if data.get(entry) is None:
                print(f'{identifier=}, {fov=}, {entry=} could not be found')
                print(f'skipping {identifier=}, {fov=}!')
                process=False
        
        if process:
            RNAs = data.get('CY5')            
            spot_data = data.get('spots')
            
            RNAs_filt=remove_background_gaussian(RNAs, sigma=sigma)
            
            spots, dense_regions, reference_spots = decompose_dense(
                RNAs_filt,
                spot_data,
                voxel_size=scale,
                spot_radius=spot_radius,
                alpha=0.5,beta=2,gamma=1
            )
            
            np.save(f"{save_path}.npy", spots)
            np.save(f"{save_path}_dd_regions.npy", dense_regions)
            io.imsave(f"{save_path}_rf_spot.tif", dense_regions)
            
            print('done')
            
        print(10*'-')